In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
import math

#!pip install sidetable
import sidetable

In [5]:
adm = pd.read_csv("./Data MIMIC/ADMISSIONS.csv")#reading csv, make your the folder "Data MIMIC" and the name "SERVICES.csv" match
print(adm.subject_id.count())
adm.head()#Define a patient’s hospital admission, 
#complete description in https://mimic.physionet.org/mimictables/admissions/

129


,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,SEPSIS,1,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,NaN,NaN,ALCOHOLIC HEPATITIS,1,1


In [3]:
patients = pd.read_csv("./Data MIMIC/PATIENTS.csv")#reading csv, make your the folder "Data MIMIC" and the name "SERVICES.csv" match
print(patients.subject_id.count())
#CALLOUT_SERVICE
patients.head()#information when a patient was READY for discharge from the ICU 
#complete description in https://mimic.physionet.org/mimictables/patients/

100


,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1


As an example, we want to join this first two tables, Admissions and patients, which has demographic information. This new table will have all values in the left db, in this case Admissions when the same subject_id appears in both tables.

In [4]:
admisions_demo = pd.merge(adm,patients,on='subject_id',how='left')
print(admisions_demo.subject_id.count())
admisions_demo.head()

129


,row_id_x,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,diagnosis,hospital_expire_flag,has_chartevents_data,row_id_y,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,SEPSIS,0,1,9467,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,...,HEPATITIS B,1,1,9472,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,...,SEPSIS,1,1,9474,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,HUMERAL FRACTURE,0,1,9478,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,...,ALCOHOLIC HEPATITIS,1,1,9479,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1


We will use properties of the imported package sidetable.
Let say we are interested in knowing the demoghaphic information of the admited as frequency of dignosis, mean age

In [10]:
#frequency table 

admisions_demo.stb.freq(['diagnosis'])


,diagnosis,count,percent,cumulative_count,cumulative_percent
0,SEPSIS,10,7.751938,10,7.751938
1,PNEUMONIA,8,6.201550,18,13.953488
2,SHORTNESS OF BREATH,4,3.100775,22,17.054264
3,FEVER,4,3.100775,26,20.155039
4,FAILURE TO THRIVE,3,2.325581,29,22.480620
...,...,...,...,...,...
90,ACUTE CHOLECYSTITIS,1,0.775194,125,96.899225
91,ACUTE CHOLANGITIS,1,0.775194,126,97.674419
92,ABSCESS,1,0.775194,127,98.449612
93,ABDOMINAL PAIN,1,0.775194,128,99.224806


In [11]:
admisions_demo.stb.freq(['gender'])


,gender,count,percent,cumulative_count,cumulative_percent
0,M,70,54.263566,70,54.263566
1,F,59,45.736434,129,100.000000


Exercise, Is it possible to calculate the age of each patient?